# SETUP

## imports

In [1]:
import string
import numpy as np
from itertools import cycle
import requests
import collections
from collections import deque
from pprint import pprint
import operator
from time import sleep
import itertools
import re
import functools
from functools import reduce
from dataclasses import dataclass
from copy import deepcopy
import networkx as nx
import matplotlib.pyplot as plt
import intcomp

## constants

In [2]:
lowercase = string.ascii_lowercase
uppercase = string.ascii_uppercase

## helpers

In [3]:
def get_level_input(lvl_num):
    with open(f"advent_inputs/{lvl_num}.txt") as f:
        level_input=f.read()
        return level_input
    
def print_result(answer):
    pprint("RESULT: "+str(answer))
    print()
    pprint("TIME"+"."*60)
    
class StopExecution(Exception):
    def _render_traceback_(self):
        pass

# 1

## setup

In [4]:
module_weights = get_level_input("01").splitlines()
module_weights = list(map(int, module_weights))

## part one

In [5]:
%%time
total_weight = sum([x//3 - 2 for x in module_weights])
print_result(total_weight)

'RESULT: 3432671'

'TIME............................................................'
CPU times: user 190 µs, sys: 92 µs, total: 282 µs
Wall time: 240 µs


## part two

In [6]:
%%time

def weight_of_weight_gen(m_weight):
    while m_weight>0:
        yield m_weight
        m_weight = m_weight//3 - 2
    
def get_req_fuel(m_weight): 
    return sum([i for i in weight_of_weight_gen(m_weight)][1:])
    
total_weight = sum([get_req_fuel(x) for x in module_weights])
print_result(total_weight)

'RESULT: 5146132'

'TIME............................................................'
CPU times: user 404 µs, sys: 35 µs, total: 439 µs
Wall time: 423 µs


# 2

## setup

In [7]:
comp_string = get_level_input("02")
comp_string = list(map(int, comp_string.split(',')))

## part one

In [8]:
%%time

comp_string_prog = comp_string[:]
comp_string_prog[1] = 12
comp_string_prog[2] = 2
res = intcomp.run_computer(comp_string_prog, [])
print_result(comp_string_prog[0])

'RESULT: 5290681'

'TIME............................................................'
CPU times: user 569 µs, sys: 256 µs, total: 825 µs
Wall time: 609 µs


## part two 

In [9]:
%%time

TARGET = 19690720
NOUN_POS = 1
VERB_POS = 2
    
def find_noun_verb(comp_string):
    for noun in range(100):
        for verb in range(100):
            comp_string_copy = comp_string[:]
            comp_string_copy[NOUN_POS] = noun
            comp_string_copy[VERB_POS] = verb
            res = intcomp.run_computer(comp_string_copy, []) 
            if comp_string_copy[0] == -1: continue 
            if(comp_string_copy[0] == TARGET): return noun,verb

noun, verb = find_noun_verb(list(comp_string))

print_result(100 * noun + verb)

'RESULT: 5741'

'TIME............................................................'
CPU times: user 822 ms, sys: 7.95 ms, total: 830 ms
Wall time: 836 ms


# 3

## setup

In [10]:
wires_input = get_level_input("03").splitlines()
wires_input = [x.split(',') for x in wires_input]

X = 1
Y = 0
CENTER_POINT = (0,0)

def md(p1, p2):
    return (abs(p1[X]-p2[X]) + abs(p1[Y]-p2[Y]))

## part one

In [11]:
%%time

def generate_wire(instructions):
    wire = [CENTER_POINT]
    for x in instructions:
        cur_point = wire[-1]
        wire += {
         'U': [(cur_point[Y]-i, cur_point[X]) for i in range(1, int(x[1:])+1)],
         'D': [(cur_point[Y]+i, cur_point[X]) for i in range(1, int(x[1:])+1)],
         'L': [(cur_point[Y], cur_point[X]-i) for i in range(1, int(x[1:])+1)],
         'R': [(cur_point[Y], cur_point[X]+i) for i in range(1, int(x[1:])+1)]
        }[x[0]]
    return wire
                                  
    
wires = [set(generate_wire(w_i))-{CENTER_POINT} for w_i in wires_input]
res = min([md(x, CENTER_POINT) for x in set.intersection(*wires)])
print_result(res)

'RESULT: 225'

'TIME............................................................'
CPU times: user 338 ms, sys: 20.7 ms, total: 359 ms
Wall time: 362 ms


## part two

In [12]:
%%time

wires = [generate_wire(w_i) for w_i in wires_input]
intersections = set.intersection(*[set(wire) for wire in wires])-{CENTER_POINT}
res = min([wires[0].index(x)+wires[1].index(x) for x in intersections])
print_result(res)

'RESULT: 35194'

'TIME............................................................'
CPU times: user 441 ms, sys: 19.2 ms, total: 460 ms
Wall time: 463 ms


# 4

## setup

In [13]:
r_low, r_high = map(int, get_level_input("04").split("-"))

def check_value(val, functions):
    num_list = [int(i) for i in list(str(val))]
    return (all(f(num_list) for f in functions))

## part one

In [14]:
%%time

def check_increase(l):
    return all(l[i] <= l[i+1] for i in range(len(l)-1))

def check_double(l):
    return any(l[i] == l[i+1] for i in range(len(l)-1))

num_poss_pass = 0
for num in range(r_low, r_high+1):
    if(check_value(num, [check_increase, check_double])): num_poss_pass+=1

print_result(num_poss_pass)

'RESULT: 1694'

'TIME............................................................'
CPU times: user 1.83 s, sys: 15.3 ms, total: 1.84 s
Wall time: 1.86 s


## part two 

In [15]:
%%time

def check_increase(l):
    return all(l[i] <= l[i+1] for i in range(len(l)-1))

def check_ungrouped_double(l):
    l = [-1] + l +[-1]
    return any(l[i-1] != l[i] == l[i+1] != l[i+2] for i in range(1, len(l)-1))

num_poss_pass = 0
for num in range(r_low, r_high+1):
    num_list = [int(i) for i in list(str(num))]
    if(check_value(num, [check_increase, check_ungrouped_double])): num_poss_pass += 1

print_result(num_poss_pass)

'RESULT: 1148'

'TIME............................................................'
CPU times: user 2.67 s, sys: 14.9 ms, total: 2.68 s
Wall time: 2.7 s


# 5

## setup

In [16]:
comp_string = get_level_input("05")
comp_string = list(map(int, comp_string.split(',')))

## part one

In [17]:
%%time

_, outputs = intcomp.run_computer(comp_string[:], [1])
print_result(outputs[-1])

'RESULT: 10987514'

'TIME............................................................'
CPU times: user 454 µs, sys: 61 µs, total: 515 µs
Wall time: 494 µs


In [18]:
%%time

_, outputs = intcomp.run_computer(comp_string[:], [5])
print_result(outputs[0])

'RESULT: 14195011'

'TIME............................................................'
CPU times: user 634 µs, sys: 90 µs, total: 724 µs
Wall time: 671 µs


# 6

## setup 

In [19]:
orbits = get_level_input("06").splitlines()
orbits = [orbit.split(')') for orbit in orbits]

## part one

In [20]:
%%time

orbit_graph = nx.DiGraph()
for orbit in orbits:
    orbit_graph.add_edge(*orbit)

orbit_checksum = sum([len(nx.descendants(orbit_graph, node)) for node in orbit_graph.nodes()])

print_result(orbit_checksum)

'RESULT: 144909'

'TIME............................................................'
CPU times: user 450 ms, sys: 7.72 ms, total: 457 ms
Wall time: 460 ms


## part two 

In [21]:
TARGET = "SAN"
START = "YOU"

orbit_graph = nx.Graph()
for orbit in orbits:
    orbit_graph.add_edge(*orbit)

START_SR = list(orbit_graph.neighbors(START))[0]
SAN_SR = list(orbit_graph.neighbors(TARGET))[0]

print_result(nx.shortest_path_length(orbit_graph, str(START_SR), str(SAN_SR)))
    

'RESULT: 259'

'TIME............................................................'


# 7

## setup 

In [22]:
amp_prog = get_level_input("07")
amp_prog = list(map(int, amp_prog.split(',')))
NUM_AMPS = 5

## part one

In [23]:
%%time

amp_pos = list(itertools.permutations(list(range(NUM_AMPS))))

max_val = 0

for perm in amp_pos:
    inputs = [perm[0], 0]
    for i in range(NUM_AMPS):
        inputs = [perm[i+1] if i+1<NUM_AMPS else 0, intcomp.run_computer(amp_prog[:], inputs)[1][0]]
        if(inputs[1]>max_val):
            max_val = inputs[1]
        
print_result(max_val)

'RESULT: 21760'

'TIME............................................................'
CPU times: user 22.1 ms, sys: 1.18 ms, total: 23.3 ms
Wall time: 22.6 ms


## part two

In [24]:
%%time

amp_pos = list(itertools.permutations(list(range(NUM_AMPS, NUM_AMPS+5))))

signals = []
for perm in amp_pos:
    i = 0
    inputs = [[perm[j]] for j in range(NUM_AMPS)]
    inputs[0].append(0)
    last_e = 0
    amp_progs = [deepcopy(amp_prog) for j in range(NUM_AMPS)]
    ips = [0 for j in range(NUM_AMPS)]
    while(True):
        ips[i%NUM_AMPS], outputs = intcomp.run_computer(amp_progs[i%NUM_AMPS], inputs[i%NUM_AMPS], ip=ips[i%NUM_AMPS], return_on_output=True)
        if outputs == []:
            break
        if(i%NUM_AMPS == 4):
            last_e = outputs[0]
        i+=1
        inputs[i%NUM_AMPS].append(outputs[0])
    signals.append(last_e)
        
print_result(max(signals))

'RESULT: 69816958'

'TIME............................................................'
CPU times: user 263 ms, sys: 3.2 ms, total: 266 ms
Wall time: 267 ms


# 8

## setup 

In [25]:
img = list(get_level_input("08"))
img = list(map(int, img))
IMG_W = 25
IMG_H = 6

## part one

In [26]:
arr_img = np.array(img)
lay_img = np.reshape(arr_img, (-1, IMG_H, IMG_W))

print(lay_img.shape)

max_zero_index = np.argmin([(lay_img[i,:,:]==0).sum() for i in range(lay_img.shape[0])])

max_zero_layer = lay_img[max_zero_index, :, :]
NUM_1_BY_2 = (max_zero_layer == 1).sum() * (max_zero_layer == 2).sum()
print(NUM_1_BY_2)

(100, 6, 25)
1360


## part two

In [27]:
BLACK = 0 
WHITE = 1 
TRANS = 2

arr_img = np.array(img)
lay_img = np.reshape(arr_img, (-1, IMG_H, IMG_W))

final_image = np.full((IMG_H, IMG_W), 2)

spears = [lay_img[:, col, row] for col in range(lay_img.shape[1]) for row in range(lay_img.shape[2])]

first_non_transparent = [spear[np.argmax(spear!=2)] for spear in spears]
message = np.array(first_non_transparent).reshape((IMG_H, IMG_W))
for row in message:
    row = np.where(row==0, '.', row) 
    row = np.where(row=='1', 'X', row) 
    print("".join(row))

XXXX.XXX..X..X..XX..XXX..
X....X..X.X..X.X..X.X..X.
XXX..X..X.X..X.X..X.X..X.
X....XXX..X..X.XXXX.XXX..
X....X....X..X.X..X.X.X..
X....X.....XX..X..X.X..X.


# 9

## setup

In [31]:
# GET LEVEL INPUT
boost_prog_raw = get_level_input("09")
boost_prog_list = list(map(int, boost_prog_raw.split(',')))
boost_prog = tuple(boost_prog_list)

## part one

In [32]:
%%time
BOOST_INPUT = 1

_, outputs = intcomp.run_computer(list(boost_prog)+[0 for x in range(9999)], [BOOST_INPUT])
print_result(outputs[0])

'RESULT: 2204990589'

'TIME............................................................'
CPU times: user 1.53 ms, sys: 122 µs, total: 1.66 ms
Wall time: 1.63 ms


## part two 

In [33]:
%%time
SENSOR_MODE = 2

_, outputs = intcomp.run_computer(list(boost_prog)+[0 for x in range(9999)], [SENSOR_MODE])
print_result(outputs[0])

'RESULT: 50008'

'TIME............................................................'
CPU times: user 1.2 s, sys: 7.05 ms, total: 1.21 s
Wall time: 1.21 s


# 10 

## setup 

In [34]:
asteroids = get_level_input("10").splitlines()
asteroid_grid = np.array([np.array(list(belt)) for belt in asteroids])
asteroid_set = frozenset(zip(*np.where(asteroid_grid == '#')[::-1]))

X = 0
Y = 1

sort_order = {
    '+': 1,
    '-': -1
}

def calc_slope(p1, p2):
    if(p1[X]-p2[X] == 0):
        return (np.inf, '+' if (p1[Y]-p2[Y]) < 0 else '-')
    if(p1[Y]-p2[Y]==0):
        return (0, '-' if (p1[X]-p2[X]) > 0 else '+')
    return ((p1[Y]-p2[Y])/(p1[X]-p2[X]), '-' if (p1[X]-p2[X]) > 0 else '+')

## part one

In [35]:
%%time

visible_sets = [(asteroid, set(map(calc_slope, itertools.repeat(asteroid), asteroid_set-{asteroid}))) for asteroid in asteroid_set]
  
monitoring_location, visible_asteroids = max(visible_sets, key = lambda x: len(x[1]))
print_result(f"{len(visible_asteroids)} asteroids visible at {monitoring_location}")

'RESULT: 284 asteroids visible at (20, 19)'

'TIME............................................................'
CPU times: user 273 ms, sys: 4.07 ms, total: 277 ms
Wall time: 277 ms


## part two 

In [36]:
%%time
MONITORING_STATION = (20, 19)
REMAINING_ASTEROIDS = asteroid_set-{MONITORING_STATION}

def l1(p1, p2):
    return abs(p1[X]-p2[X])+abs(p1[Y]-p2[Y])

def calc_slope_and_dist(p1, p2):
    dist = l1(p1,p2)
    if(p1[X]-p2[X] == 0):
        return (np.inf, '+' if (p1[Y]-p2[Y]) > 0 else '-', dist)
    if(p1[Y]-p2[Y]==0):
        return (0, '-' if (p1[X]-p2[X]) > 0 else '+', dist)
    return (-(p1[Y]-p2[Y])/(p1[X]-p2[X]), '-' if (p1[X]-p2[X]) > 0 else '+', dist)

visible_list = [(asteroid, calc_slope_and_dist(MONITORING_STATION, asteroid)) for asteroid in REMAINING_ASTEROIDS]
visible_list.sort(key=lambda k: (sort_order[k[1][1]], k[1][0], -k[1][2]), reverse=True)

visible_stacked = [list(g) for k, g in itertools.groupby(visible_list, key=lambda k: k[1][:1])]
two_hundredth_vape = visible_stacked[199][0][0]

output_val = two_hundredth_vape[0]*100+two_hundredth_vape[1]

print_result(output_val)

'RESULT: 404'

'TIME............................................................'
CPU times: user 3.6 ms, sys: 738 µs, total: 4.34 ms
Wall time: 3.7 ms


# 11 

## setup 

In [60]:
paint_prog = get_level_input("11")
paint_prog = list(map(int, paint_prog.split(',')))
paint_prog = tuple(paint_prog)